In [739]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [740]:
train = pd.read_csv('../input/robi-datathon-2-pre-assessment/train.csv')
test = pd.read_csv('../input/robi-datathon-2-pre-assessment/test.csv')
raw_train = pd.read_csv('../input/robi-datathon-2-pre-assessment/train.csv')
raw_test = pd.read_csv('../input/robi-datathon-2-pre-assessment/test.csv')

In [741]:
print (train.info())

In [742]:
#Dropping empty column
train = train.drop(['s56', 's57', 's59', "s54", "s55", "s53", 'id'], axis = 1)
test = test.drop(['s56', 's57', 's59', "s54", "s55", "s53", 'id'], axis = 1)

In [743]:
train.label.count()

In [744]:
print(train.groupby(['label'])['label'].count())

Out of 28322 people 4760 people bought the product. About 16.8% bought the product.

In [745]:
train[train['gender']=="F"].groupby(['gender','label']).size().reset_index(name='count')


Only 276 Female have bought the product. That means if female there is high possibility of not buying the product. Thats why we keep this column. Men have higher tendency of buying the product.

In [746]:
train[train['s11']=="N"].groupby(['s11','label']).size().reset_index(name='count')

In [747]:
print (train[['s11', 'label']].groupby(['s11'], as_index=False).mean())

In [748]:
print (train[['s12', 'label']].groupby(['s12'], as_index=False).mean())

In [749]:
train[train['s13']==0].groupby(['s13','label']).size().reset_index(name='count')

In [750]:
print (train[['s13', 'label']].groupby(['s13'], as_index=False).mean())

76% of people who were 0 in s13 column bought. useful insight

In [751]:
print (train[['s16', 'label']].groupby(['s16'], as_index=False).mean())

In [752]:
train[train['s16']=="B"].groupby(['s16','label']).size().reset_index(name='count')

In [753]:
#Fill Empty value with "D" becuase it has highest count
train["s17"] = train['s17'].fillna('D')

In [754]:
print (train[['s17', 'label']].groupby(['s17'], as_index=False).mean())

In [755]:
train[train['s18']=="D"].groupby(['s18','label']).size().reset_index(name='count')

In [756]:
print (train[['s18', 'label']].groupby(['s18'], as_index=False).mean())

In [757]:
print (train[['s48', 'label']].groupby(['s48'], as_index=False).mean())

In [758]:
print (train[['s52', 'label']].groupby(['s52'], as_index=False).mean())

In [759]:
print (train[['s58', 'label']].groupby(['s58'], as_index=False).mean())

In [760]:
print (train[['s69', 'label']].groupby(['s69'], as_index=False).mean())

In [761]:
print (train[['s70', 'label']].groupby(['s70'], as_index=False).mean())

In [762]:
print (train[['s71', 'label']].groupby(['s71'], as_index=False).mean())

In [763]:
train[train['s71']=="c"].groupby(['s71','label']).size().reset_index(name='count')

In [764]:
train.n1.describe()

In [765]:
train['Categoricaln1'] = pd.qcut(train['n1'], 7)
print(train[['Categoricaln1', 'label']].groupby(['Categoricaln1'], as_index=False).mean())

In [766]:
train.n2.describe()

In [767]:
train['Categoricaln2'] = pd.qcut(train['n2'], 7)
print(train[['Categoricaln2', 'label']].groupby(['Categoricaln2'], as_index=False).mean())

In [768]:
train.n3.describe()

In [769]:
print (train[['n3', 'label']].groupby(['n3'], as_index=False).mean())

In [770]:
train.n4.describe()

In [771]:
train['Categoricaln4'] = pd.qcut(train['n4'], 7)
print(train[['Categoricaln4', 'label']].groupby(['Categoricaln4'], as_index=False).mean())

In [772]:
train.n5.describe()

In [773]:
train['Categoricaln5'] = pd.qcut(train['n5'], 7)
print(train[['Categoricaln5', 'label']].groupby(['Categoricaln5'], as_index=False).mean())

In [774]:
train['Categoricaln6'] = pd.qcut(train['n6'], 7)
print(train[['Categoricaln6', 'label']].groupby(['Categoricaln6'], as_index=False).mean())

In [775]:
train['Categoricaln7'] = pd.qcut(train['n7'], 7)
print(train[['Categoricaln7', 'label']].groupby(['Categoricaln7'], as_index=False).mean())

In [776]:
train['Categoricaln8'] = pd.qcut(train['n8'], 7)
print(train[['Categoricaln8', 'label']].groupby(['Categoricaln8'], as_index=False).mean())

In [777]:
train['Categoricaln9'] = pd.qcut(train['n9'], 7)
print(train[['Categoricaln9', 'label']].groupby(['Categoricaln9'], as_index=False).mean())

In [778]:
train['Categoricaln10'] = pd.qcut(train['n10'], 7)
print(train[['Categoricaln10', 'label']].groupby(['Categoricaln10'], as_index=False).mean())

In [779]:
train['Categoricaln11'] = pd.qcut(train['n11'], 7)
print(train[['Categoricaln11', 'label']].groupby(['Categoricaln11'], as_index=False).mean())

In [780]:
train['Categoricaln14'] = pd.qcut(train['n14'], 7)
print(train[['Categoricaln14', 'label']].groupby(['Categoricaln14'], as_index=False).mean())

In [781]:
full_data = [train, test]

In [782]:
for dataset in full_data:
    dataset['gender'] = dataset['gender'].map( {'F': 0, 'M': 1} ).astype(int)
    
    dataset['s11'] = dataset['s11'].map( {'Y': 0, 'N': 1} ).astype(int)
    dataset['s12'] = dataset['s12'].map( {'Y': 0, 'N': 1} ).astype(int)
    
    dataset['s16'] = dataset['s16'].map( {'A': 0, 'B': 1, 'C': 2, 'D': 3} ).astype(int)
    dataset['s17'] = dataset['s17'].map( {'A': 0, 'B': 1, 'C': 2, 'D': 3} ).astype(int)
    dataset['s18'] = dataset['s18'].map( {'A': 0, 'B': 1, 'C': 2, 'D': 3} ).astype(int)
    
    dataset['s52'] = dataset['s52'].map( {'0': 0, '1':1, 'l': 2, 'o': 3} ).astype(int)
    dataset['s58'] = dataset['s58'].map( {'A': 0, 'B': 1} ).astype(int)
    
    dataset['s69'] = dataset['s69'].map( {'0': 0, '~1': 1, 'C`': 2, 'x': 3} ).astype(int)
    dataset['s70'] = dataset['s70'].map( {'op: A': 0, 'op: B': 1, 'op: C': 2, 'op: D': 3} ).astype(int)
    dataset['s71'] = dataset['s71'].map( {'a': 0, 'b': 1, 'c': 2, 'd': 3} ).astype(int)
    
    dataset.loc[(dataset['n1'] >= 1.5) & (dataset['n1'] <= 5.2), 'n1'] = 0
    dataset.loc[(dataset['n1'] > 5.2) & (dataset['n1'] <= 6.8), 'n1'] = 1
    dataset.loc[(dataset['n1'] > 6.8) & (dataset['n1'] <= 8.5), 'n1'] = 2
    dataset.loc[(dataset['n1'] > 8.5) & (dataset['n1'] <= 10.5), 'n1'] = 3
    dataset.loc[(dataset['n1'] > 10.5) & (dataset['n1'] <= 13.4), 'n1'] = 4
    dataset.loc[(dataset['n1'] > 13.4) & (dataset['n1'] <= 15.7), 'n1'] = 5
    dataset.loc[(dataset['n1'] > 15.7) & (dataset['n1'] <= 22), 'n1'] = 6
    
    dataset.loc[(dataset['n2'] >= 0) & (dataset['n2'] <= 0.87), 'n2'] = 0
    dataset.loc[(dataset['n2'] > 0.87) & (dataset['n2'] <= 1.065), 'n2'] = 1
    dataset.loc[(dataset['n2'] > 1.065) & (dataset['n2'] <= 1.361), 'n2'] = 2
    dataset.loc[(dataset['n2'] > 1.361) & (dataset['n2'] <= 1.63), 'n2'] = 3
    dataset.loc[(dataset['n2'] > 1.63) & (dataset['n2'] <= 2.095), 'n2'] = 4
    dataset.loc[(dataset['n2'] > 2.095) & (dataset['n2'] <= 2.452), 'n2'] = 5
    dataset.loc[(dataset['n2'] > 2.452) & (dataset['n2'] <= 4), 'n2'] = 6
     
    dataset.loc[(dataset['n4'] >= 1.6) & (dataset['n4'] <= 2.37), 'n4'] = 0
    dataset.loc[(dataset['n4'] > 2.37) & (dataset['n4'] <= 2.92), 'n4'] = 1
    dataset.loc[(dataset['n4'] > 2.92) & (dataset['n4'] <= 3.82), 'n4'] = 2
    dataset.loc[(dataset['n4'] > 3.82) & (dataset['n4'] <= 6.06), 'n4'] = 3
    dataset.loc[(dataset['n4'] > 6.06) & (dataset['n4'] <= 7.13), 'n4'] = 4
    dataset.loc[(dataset['n4'] > 7.13) & (dataset['n4'] <= 7.96), 'n4'] = 5
    dataset.loc[(dataset['n4'] > 7.96) & (dataset['n4'] <= 9), 'n4'] = 6
    
    dataset.loc[(dataset['n5'] >= -34.16) & (dataset['n5'] <= -32.97), 'n5'] = 0
    dataset.loc[(dataset['n5'] > -32.97) & (dataset['n5'] <= -32.84), 'n5'] = 1
    dataset.loc[(dataset['n5'] > -32.84) & (dataset['n5'] <= -32.64), 'n5'] = 2
    dataset.loc[(dataset['n5'] > -32.64) & (dataset['n5'] <= -32.57), 'n5'] = 3
    dataset.loc[(dataset['n5'] > -32.57) & (dataset['n5'] <= -32.44), 'n5'] = 4
    dataset.loc[(dataset['n5'] > -32.44) & (dataset['n5'] <= -32.36), 'n5'] = 5
    dataset.loc[(dataset['n5'] > -32.36) & (dataset['n5'] <= -30.29), 'n5'] = 6
    
    dataset.loc[(dataset['n6'] >= -0.0004) & (dataset['n6'] <= 0.006), 'n6'] = 0
    dataset.loc[(dataset['n6'] > 0.006) & (dataset['n6'] <= 0.01), 'n6'] = 1
    dataset.loc[(dataset['n6'] > 0.01) & (dataset['n6'] <= 0.0148), 'n6'] = 2
    dataset.loc[(dataset['n6'] > 0.0148) & (dataset['n6'] <= 0.0169), 'n6'] = 3
    dataset.loc[(dataset['n6'] > 0.0169) & (dataset['n6'] <= 0.0183), 'n6'] = 4
    dataset.loc[(dataset['n6'] > 0.0183) & (dataset['n6'] <= 0.022), 'n6'] = 5
    dataset.loc[(dataset['n6'] > 0.022) & (dataset['n6'] <= 0.04), 'n6'] = 6
    
    dataset.loc[(dataset['n7'] >= -9.519) & (dataset['n7'] <= -9.293), 'n7'] = 0
    dataset.loc[(dataset['n7'] > -9.293) & (dataset['n7'] <= -9.249), 'n7'] = 1
    dataset.loc[(dataset['n7'] > -9.249) & (dataset['n7'] <= -9.217), 'n7'] = 2
    dataset.loc[(dataset['n7'] > -9.217) & (dataset['n7'] <= -9.116), 'n7'] = 3
    dataset.loc[(dataset['n7'] > -9.116) & (dataset['n7'] <= -9.01), 'n7'] = 4
    dataset.loc[(dataset['n7'] > -9.01) & (dataset['n7'] <= -8.825), 'n7'] = 5
    dataset.loc[(dataset['n7'] > -8.825) & (dataset['n7'] <= -8.0), 'n7'] = 6
    
    dataset.loc[(dataset['n8'] >= 1) & (dataset['n8'] <= 1.469), 'n8'] = 0
    dataset.loc[(dataset['n8'] > 1.469) & (dataset['n8'] <= 1.495), 'n8'] = 1
    dataset.loc[(dataset['n8'] > 1.495) & (dataset['n8'] <= 1.518), 'n8'] = 2
    dataset.loc[(dataset['n8'] > 1.518) & (dataset['n8'] <= 1.575), 'n8'] = 3
    dataset.loc[(dataset['n8'] > 1.575) & (dataset['n8'] <= 1.704), 'n8'] = 4
    dataset.loc[(dataset['n8'] > 1.704) & (dataset['n8'] <= 1.829), 'n8'] = 5
    dataset.loc[(dataset['n8'] > 1.829) & (dataset['n8'] <= 2.5), 'n8'] = 6
    
    dataset.loc[(dataset['n9'] >= 2) & (dataset['n9'] <= 3.069), 'n9'] = 0
    dataset.loc[(dataset['n9'] > 3.069) & (dataset['n9'] <= 3.568), 'n9'] = 1
    dataset.loc[(dataset['n9'] > 3.568) & (dataset['n9'] <= 4.0), 'n9'] = 2
    dataset.loc[(dataset['n9'] > 4.0) & (dataset['n9'] <= 5.202), 'n9'] = 3
    dataset.loc[(dataset['n9'] > 5.202) & (dataset['n9'] <= 6.238), 'n9'] = 4
    dataset.loc[(dataset['n9'] > 6.238) & (dataset['n9'] <= 8.131), 'n9'] = 5
    dataset.loc[(dataset['n9'] > 8.131) & (dataset['n9'] <= 11.9), 'n9'] = 6
    
    dataset.loc[(dataset['n10'] >= 1) & (dataset['n10'] <= 3.878), 'n10'] = 0
    dataset.loc[(dataset['n10'] > 3.878) & (dataset['n10'] <= 4.59), 'n10'] = 1
    dataset.loc[(dataset['n10'] > 4.59) & (dataset['n10'] <= 4.92), 'n10'] = 2
    dataset.loc[(dataset['n10'] > 4.92) & (dataset['n10'] <= 5.623), 'n10'] = 3
    dataset.loc[(dataset['n10'] > 5.623) & (dataset['n10'] <= 6.48), 'n10'] = 4
    dataset.loc[(dataset['n10'] > 6.48) & (dataset['n10'] <= 9.841), 'n10'] = 5
    dataset.loc[(dataset['n10'] > 9.841) & (dataset['n10'] <= 12.9), 'n10'] = 6
    
    dataset.loc[(dataset['n11'] >= 1.498) & (dataset['n11'] <= 1.558), 'n11'] = 0
    dataset.loc[(dataset['n11'] > 1.558) & (dataset['n11'] <= 1.615), 'n11'] = 1
    dataset.loc[(dataset['n11'] > 1.615) & (dataset['n11'] <= 1.67), 'n11'] = 2
    dataset.loc[(dataset['n11'] > 1.67) & (dataset['n11'] <= 1.727), 'n11'] = 3
    dataset.loc[(dataset['n11'] > 1.727) & (dataset['n11'] <= 1.785), 'n11'] = 4
    dataset.loc[(dataset['n11'] > 1.785) & (dataset['n11'] <= 1.843), 'n11'] = 5
    dataset.loc[(dataset['n11'] > 1.843) & (dataset['n11'] <= 1.99), 'n11'] = 6    
    
    dataset.loc[(dataset['n14'] >= 0) & (dataset['n14'] <= 0.138), 'n14'] = 0
    dataset.loc[(dataset['n14'] > 0.138) & (dataset['n14'] <= 0.284), 'n14'] = 1
    dataset.loc[(dataset['n14'] > 0.284) & (dataset['n14'] <= 0.425), 'n14'] = 2
    dataset.loc[(dataset['n14'] > 0.425) & (dataset['n14'] <= 0.566), 'n14'] = 3
    dataset.loc[(dataset['n14'] > 0.566) & (dataset['n14'] <= 0.712), 'n14'] = 4
    dataset.loc[(dataset['n14'] > 0.712) & (dataset['n14'] <= 0.858), 'n14'] = 5
    dataset.loc[(dataset['n14'] > 0.858) & (dataset['n14'] <= 1.09), 'n14'] = 6

In [783]:
train.head()

In [784]:
train = train.drop(['Categoricaln1', 'Categoricaln2', 'Categoricaln4', 'Categoricaln5', 'Categoricaln6', 'Categoricaln7', 'Categoricaln8', 'Categoricaln9', 'Categoricaln10', 'Categoricaln11', 'Categoricaln14'], axis = 1)

In [785]:
corrmat = train.corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'label')['label'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [786]:
train["label"] = train["label"].astype(int)

Y_train = train["label"]

X_train = train.drop(labels = ["label"],axis = 1)

In [787]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [788]:
# Modeling step Test differents algorithms 
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

In [789]:
### META MODELING  WITH ADABOOST, RF, EXTRATREES and GRADIENTBOOSTING

# Adaboost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train,Y_train)

ada_best = gsadaDTC.best_estimator_

In [790]:
gsadaDTC.best_score_

In [791]:
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train,Y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [792]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train,Y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [794]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train,Y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

In [797]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, Y_train)

In [798]:
test_Survived = pd.Series(votingC.predict(test), name="label")

In [799]:
test_id = raw_test["id"]

In [800]:
results = pd.concat([test_id,test_Survived],axis=1)
results.to_csv("ensemble_python_voting.csv",index=False)